In [ ]:
import pandas as pd
import pypff
import os
import requests
import json
from pandas import ExcelWriter
from bs4 import BeautifulSoup
from os import listdir
from os.path import isfile, isdir, join

#====================================================================#

# JIRA 帳號密碼
user='J0278'
pwd='qazwsX1324'

# 1000 X 5 的 array 用來存資料
list = [[None]*5 for z in range(1000)]
temp=[0]*50
# jira 填資料用的
to_components=[]
to_divsion=[]
to_who=[]

def main():
    mypath = "D:/mail"   #=======要改路徑=======#
    files = listdir(mypath)

    for f in files:    # f==> 在資料夾內的所有 pst 檔案名稱

        opst = pypff.open(f) #=======要改路徑=======#
        root = opst.get_root_folder()
        createFolder('./你好安安') #=======要改路徑=======#
        sorting_mail(root)
        CREATE(list,temp)
        jira(user,pwd,temp)
        # 清空
        to_components=[]
        to_divsion=[]
        to_who=[]
    
def sorting_mail(root):
    z=0
    # 一層一層找
    for i in root.sub_folders:
        for j in i.sub_folders:
            for k in j.sub_folders:
                for message in k.sub_messages:# 第四層
                    
                    contents= message.plain_text_body.decode("utf-8") # 內容
    
                    subject=message.subject          # 標題
                    head=subject.find('"')           # 找 " " 裡中間的名稱 第一個 "
                    tail=subject.find('"',head+1)    # 第二個 "
                    subject=subject[head+1:tail]     # " "的範圍內
                    
                    html=str(message.html_body)      # 網址
                    head=html.find('href="')         # 找 href=" " 裡中間的網址 第一個 "
                    tail=html.find('"',head+6)       # 第二個 "
                    html=html[head+6:tail]           # " "的範圍內
                    
                    save_to_list(str(message.transport_headers),z,message.sender_name,subject
                                 ,list,contents,html)
                    z=z+1                     
    return list

def save_to_list(time,z,sender,subject,list,contents,html):
    # 寄件時間的範圍
    place=time.find(';')
    place2=time.find('+')
    # 把可能會跳行的改成不跳行
    Time=time[place+1:place2-1]
    Time=Time.replace("\r","")
    Time=Time.replace("\n","")
    # 存入 list
    list[z][0]=sender
    list[z][1]=subject
    list[z][2]=Time
    list[z][3]=contents
    list[z][4]=html
    
    return list
    
def CREATE(list,temp):
    df=pd.DataFrame(list,columns=['寄件人','標題','寄件時間','內容','url'])
    createExcel(df[0:1],list[0][1]) # 直接創excel a
    temp[0]=list[0][1]  # 暫存
    for a in range(1,5):                                    #=======要改數字=======#
        if list[a][1] in temp:
            appendExcel(df[a:a+1],list[a][1])
        else:
            createExcel(df[a:a+1],list[a][1])
            temp[a]=list[a][1]
    return temp

def createExcel(df,subject):
    excel=pd.ExcelWriter('你好安安/{}.xlsx'.format(subject)) #=======要改路徑=======#
    df.to_excel(excel)
    excel.save()
def appendExcel(df2,subject):
    df1 = pd.read_excel('你好安安/{}.xlsx'.format(subject)) #=======要改路徑=======#
    df3=pd.concat([df1,df2])
    
    excel=pd.ExcelWriter('你好安安/{}.xlsx'.format(subject)) #=======要改路徑=======#
    df3.to_excel(excel)
    excel.save()
    
def createFolder(directory):
        try:
            if not os.path.exists(directory):
                os.makedirs(directory)
        except OSError:
            print ('Error: Creating directory. ' +  directory)    
            
def jira(user,pwd,temp,to_components,to_divsion,to_who):
    url = "https://km.fubonlife.com.tw/jira/rest/api/2/issue"
    systemData(temp,user,pwd,to_components,to_divsion,to_who)
    
    string="數據暨智能發展科" #請選擇系統維護單位 要改
    for g in range (500):
        postdata={
            "fields": {
                "project": {"key": "ITSYS"},
                "issuetype": {"id": "3"},
                "customfield_10831": {"value":"資訊服務處運營管理部"}, #請選擇報案單位
                "components":[{"id":"11203"}],  # 要改
                "customfield_10820": {"value":string}, #請選擇系統維護單位 要改
                "summary": "test test", #要改
                "customfield_10808":[{"name":"X1493, J0277,"}] #要改
            }
        }

        response = requests.post(url, json=(postdata) ,auth=(user,pwd))

        issue_id = response.json()['id']
        url = "https://km.fubonlife.com.tw/jira/rest/api/2/issue/{}/attachments".format(issue_id)
        files = {'file': open('SAV_Home.xlsx', 'rb')} #=======要改路徑=======#
        headers = {"X-Atlassian-Token": "nocheck"}
        response = requests.post(url, auth=(user,pwd), files=files, headers=headers)


def systemData(temp,user,pwd,to_components,to_divsion,to_who):
    url='https://km.fubonlife.com.tw/confluence/pages/viewpage.action?pageId=37432468'
    response=requests.get(url,auth=(user,pwd))
    
    soup = BeautifulSoup(response.content,'lxml')
    table = soup.find_all('table')[0] # 整份表格
    table_rows = table.find_all('tr')

    l=[]
    for tr in table_rows:
        td = tr.find_all('td')
        row = [tr.text for tr in td]# 一列一列
        l.append(row)

    df=pd.DataFrame(l,columns=[1,2,3,4,5,6,7,8,9,10,11,12,13])
    df1 = df[[4,5,8,12]]
    df1=df1.drop([0]) #把空行刪除
    
    check(df1,temp,to_components,to_divsion,to_who)
    return to_components,to_divsion,to_who
    
def check(df1,temp,to_components,to_divsion,to_who):#string=df1.iloc[[j], [0]]

    df_check=df1.set_index([4,5,8,12])
    
    x=len(temp)
    for i in range(x):
        tocheck=str(temp[i])
        a=tocheck.find("_")
        tocheck=tocheck[0:a]
        if tocheck in df_check.index: # tocheck 有沒有在 df_check裡面

            x=df1[df1[4]==tocheck][5].iloc[0]
            y=df1[df1[4]==tocheck][8].iloc[0]
            z=df1[df1[4]==tocheck][12].iloc[0]
            
            to_components.append(x)
            to_divsion.append(y)
            to_who.append(z)
            
    return to_components,to_divsion,to_who
    
    
if __name__ == '__main__':
    main()

In [2]:
opst = pypff.open('123.pst') 
root = opst.get_root_folder()

for i in root.sub_folders:
        for message in i.sub_messages:
            print(message.sender_name,'\t',message.subject)

陳珮綺 	 ALERT: AppInternals - "SAV_Policy List" : "Server Response Time"
陳珮綺 	 ALERT: AppInternals - "SAV_Login" : "Server Response Time"
陳珮綺 	 ALERT: AppInternals - "SAV_Home" : "Server Response Time"
陳珮綺 	 ALERT: AppInternals - "SAV_Policy List" : "Server Response Time"
